**python chunking method is faster than pandas** <br>
On a 3 gb file with 11.8 million rows of data ,comparatively it takes 9 seconds with python and 160 secs with pandas

In [16]:
batch_size=40000

def write_chunk(part_id , lines):
     with open("C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\1995_part"+str(part_id)+".csv" , 'w') as fw:
          fw.write(header);
          fw.writelines(lines);
     

with open("C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\1995\\1995_data.csv", "r") as f:
    count = 0
    lines = []
    header = f.readline() #this will repeat in  all files
    for l in f:
        lines.append(l)
        count += 1 #loop first 40000 lines and accumulate the counter till 40000 to be divisible by it
        if count % batch_size == 0 : #first 40k, next 80k and so on
            write_chunk(count // batch_size , lines )   # part file number (floor division) and lines list
            lines = [] #reset in order to hold next set of lines
    
    if len(lines) > 0:
          # we need this as the count won't be divisible by the batch size
          write_chunk((count // batch_size) + 1, lines ) #the remaining part ,say 26000


In [19]:
#using pandas to enumerate thru chunks of data and loading into small files or even to a table

import pandas as pd
#from snowflake.connector.pandas_tools import write_pandas

source_path = "C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\1995\\1995_data.csv"
for i,chunk in enumerate(pd.read_csv(source_path, chunksize=40000)):
    chunk.to_csv('C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\chunk{}.csv'.format(i), index=False)

# for i,chunk in enumerate(pd.read_csv(source_path, chunksize=40000)):
#     write_pandas(conn,df,table,database,schema)    

In [15]:
import pandas as pd

source_path = "C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\1995\\1995_data.csv"

for idx,chunk in enumerate(pd.read_csv(source_path , chunksize = 40000)):
    chunk.to_csv(f'C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\part{idx}.csv' , index=False)

In [25]:

from io import BytesIO
import zipfile
source_path = "C:\\Users\\conne\\snowflake\\files\\load\\s3_data\\1995\\1995_data.csv"
buf = BytesIO(open(source_path,"rb").read(8192));
zipped = zipfile.ZipFile(buf)
zipped.namelist()